<h1>Содержание<span class="tocSkip"></span></h1>
<li><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li>

# Классификация комментариев

**Описание данных:**
    
Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

**Цель исследования:**
    
Построить модель, классифицирующую комментарии, со значением метрики качества F1 не меньше 0.75.

**Ход исследования:**
1. Подготовка данных
2. Обучение модели
3. Вывод по исследованию

## Подготовка

In [1]:
# Импорт библиотек
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.metrics import f1_score,make_scorer
import transformers
import numpy as np
from sklearn.model_selection import cross_val_score
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

In [2]:
#Сохраним данные в переменную data
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
#Посмотрим первые пять строк
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
#Посмотрим общую информацию
data.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 159292 entries, 0 to 159291

Data columns (total 3 columns):

 #   Column      Non-Null Count   Dtype 

---  ------      --------------   ----- 

 0   Unnamed: 0  159292 non-null  int64 

 1   text        159292 non-null  object

 2   toxic       159292 non-null  int64 

dtypes: int64(2), object(1)

memory usage: 3.6+ MB


In [5]:
#Посмотрим наличие дубликатов
data.duplicated().sum()

0

Дубликаты отсутствуют

In [6]:
#Проверим уникальные значения столбца toxic
data.toxic.value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Присутствует явный дисбаланс

In [7]:
#Переведём столбец text в нижний регистр
data['text'] = data['text'].str.lower()

In [8]:
#Очистим текст
def clean(text):
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    return text

data['text'] = data['text'].apply(clean)

In [9]:
#Проверим очистку
print(data.text[:2])

0    explanation why the edits made under my userna...

1    daww he matches this background colour im seem...

Name: text, dtype: object


In [11]:
#Лемматизируем текст и создадим новый столбец
def lemmetize(words):
    lemmatizer = WordNetLemmatizer() 
    text = []
    tokens = word_tokenize(words)
    for token in tokens:
        lem_word = lemmatizer.lemmatize(token) 
        text.append(lem_word)
    sentence = " ".join(text)
    return sentence

data['lem_text'] = data['text'].apply(lemmetize)

## Обучение

In [12]:
#Разделим данные на обучающую, валидационную и тестовую выборки
target = data['toxic']
features = data['lem_text']

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345) 

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.50, random_state=12345)

In [13]:
#Посмотрим размеры выборок
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(95575,)

(95575,)

(31858,)

(31858,)

(31859,)

(31859,)


In [14]:
#Загрузим список стоп-слов
stopwords = set(nltk_stopwords.words('english'))
#Создадим счётчик стоп-слов
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [15]:
#Переведём выборки в векторы и посчитаем TF-IDF
train_tf_idf = count_tf_idf.fit_transform(features_train)
valid_tf_idf = count_tf_idf.transform(features_valid)
test_tf_idf = count_tf_idf.transform(features_test)

In [16]:
#Посмотрим размеры матриц
print(train_tf_idf.shape)
print(valid_tf_idf.shape)
print(test_tf_idf.shape)

(95575, 150920)

(31858, 150920)

(31859, 150920)


In [17]:
#Обучим модель логистической регрессии
model_lr = LogisticRegression(class_weight='balanced', random_state=12345, max_iter=1000, C=5)
 
model_lr.fit(train_tf_idf, target_train)
pred_valid_lr = model_lr.predict(valid_tf_idf)
print("F1_score:", f1_score(target_valid, pred_valid_lr))

F1_score: 0.7620108619969365


In [18]:
#Обучим модель случайного леса
for depth in range(3, 20, 3):
    model = RandomForestClassifier(random_state=12345, 
                                   max_depth = depth, 
                                   class_weight='balanced')
    model.fit(train_tf_idf, target_train)
    pred_valid_forest = model.predict(valid_tf_idf)
    print('Глубина:', depth)
    print("F1_score:", f1_score(target_valid, pred_valid_forest))    
    print('')
print()

Глубина: 3

F1_score: 0.32237835326907444



Глубина: 6

F1_score: 0.36755053819900235



Глубина: 9

F1_score: 0.3690249819683955



Глубина: 12

F1_score: 0.36892816798458367



Глубина: 15

F1_score: 0.38169304886441846



Глубина: 18

F1_score: 0.39096462565705353






Логистическая регрессия показала хороший результат, протестируем её

In [21]:
#Протестируем модель
model_lr = LogisticRegression(class_weight='balanced', random_state=12345, max_iter=1000, C=5)
model_lr.fit(train_tf_idf, target_train)
pred_test_lr = model_lr.predict(test_tf_idf)
print("F1_score:", f1_score(target_test, pred_test_lr))

F1_score: 0.7537109093529327


## Выводы

* Данные были загружены и подготовлены для обучения
    * Проведена очистка текста
    * Проведена лемматизация текста
    * Значения столбца text приведены к нижнему регистру
* Данные разделили на признаки и целевой признак
* Обучены две модели: Логистическая регрессия и случайный лес
* По результатам обучения для тестирования была выбрана модель "Логистическая регрессия" с результатом f1-метрики - 0.76
* На тестовой выборке модел показала значение 0.75, что соответствует требованиям